# Simple Reflex Agent
Los simple reflex agents son programas que toman decisiones basadas únicamente en la información actual que reciben del entorno (percept), sin considerar el historial de percepciones pasadas. Funcionan siguiendo reglas de tipo "if-then" (si-entonces) para reaccionar de manera inmediata a los estímulos, pero no pueden planificar ni aprender de experiencias previas.

In [24]:
!python -m venv venv || source venvnb2\Scripts\activate
!pip install ollama random

Unable to copy 'C:\\Users\\Bo\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\venv\\scripts\\nt\\venvlauncher.exe' to 'c:\\gitprojects\\ses10\\venv\\Scripts\\python.exe'


ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


En el ejemplo siguiente, se implementa un agente reflexivo simple que toma decisiones basadas únicamente en la percepción actual del entorno.

El agente recibe como entrada un diccionario con valores de temperatura y presión, y utiliza un conjunto de reglas de tipo "si-entonces" para determinar la acción a realizar (por ejemplo, encender o apagar la calefacción, activar o desactivar una bomba).

El agente evalúa cada regla y ejecuta la acción correspondiente si se cumple la condición, sin considerar el historial de percepciones previas ni mantener un modelo interno del entorno.

In [25]:
import random
import ollama

# Reglas de condición-acción
reglas = {
    "temperatura < 18": "encender calefacción",
    "temperatura > 24": "apagar calefacción",
    "presión < 30": "activar bomba",
    "presión > 70": "desactivar bomba"
}

In [26]:
# Función para evaluar condiciones simples
def cumple_condicion(percepcion, condicion):
    variable, operador, valor = condicion.split()
    valor = float(valor)
    entrada = percepcion.get(variable)

    if entrada is None:
        return False

    if operador == "<":
        return entrada < valor
    elif operador == ">":
        return entrada > valor
    elif operador == "==":
        return entrada == valor
    else:
        return False

# Simple Reflex Agent
def agente_reflejo_simple(percepcion):
    for condicion, accion in reglas.items():
        if cumple_condicion(percepcion, condicion):
            print(f"Acción local: {accion}")
            return accion

    print("Sin acción necesaria.")
    return "sin acción"


In [27]:
# Ejemplo de uso
percepcion_actual = {
    "temperatura": random.randint(10, 35), #15 in Celsius
    "presión": random.randint(20, 80) #45 in PSI
}
print(f"Percepción actual: {percepcion_actual}")
accion_tomada = agente_reflejo_simple(percepcion_actual)


Percepción actual: {'temperatura': 18, 'presión': 42}
Sin acción necesaria.


# Model Based Reflex Agent
Un Model Based Reflex Agent es un agente que, además de reaccionar a los estímulos actuales (percepts), mantiene un modelo interno (model) del estado del entorno. Esto le permite tomar decisiones considerando no solo la percepción inmediata, sino también información sobre cómo el entorno puede cambiar con el tiempo. Así, puede manejar situaciones donde la percepción actual no es suficiente para decidir la acción correcta, usando su modelo para inferir el estado real del entorno.

In [28]:
!pip install langchain langchain_ollama langchain-openai python-dotenv

In [29]:
from langchain.chains.llm import LLMChain
from langchain_ollama.llms import OllamaLLM
from langchain.memory.buffer import ConversationBufferMemory
from langchain_core.prompts.chat import ChatPromptTemplate

from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

# Configuración del LLM local con Ollama y modelo llama3.2
llm = OllamaLLM(model="llama3.2")

system_prompt = """Eres asistente inteligente de banca y finanzas.
Que decide la mejor acción en base a el estado actual de tu cliente.
Las acciones que puedes tomar son:
- mostrar saldo de cuenta ** esta no requiere confirmación del cliente **
- transferir dinero ** esta requiere confirmación del cliente **
- pagar factura ** esta requiere confirmación del cliente **
- invertir ** esta requiere confirmación del cliente **
** cuando estes a punto de realizar una acción, primero pregunta si el cliente desea proceder con la acción. De esa forma evitas acciones no deseadas y recoges el feedback del cliente. **
Tomas en cuenta tu historial de de tus acciones y confirmas si tu cliente decidio proceder o no con la accion que sugieras.
- cuando identifiques la accion a tomar, responde ademas con la accion en minusculas y sin puntuacion. Si no hay ninguna accion a tomar, responde "ninguna".
    por ejemplo:
    Accion tomada por agente: mostrar saldo de cuenta
- cuando identifiques el feedback del cliente, responde ademas con el feedback en minusculas y sin puntuacion.
    Por ejemplo:
    Feedback del cliente: si, deseo proceder con la accion
"""

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])


In [30]:
pipeline = prompt_template | llm 

In [31]:
import re
import json

def search_json_in_text(text):
    # Expresión regular para encontrar bloques JSON
    pattern = r'\{.*?\}'
    matches = re.findall(pattern, text)
    return json.loads(matches)

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
state_map = {}

class State:
    def __init__(self, query, status):
        self.input = user_query
        self.action = agent_action
        self.feedback = user_feedback

feedback_prompt = (
            """Analiza el historico de conversacion: {history}
            Ahora devuelve siguientes campos y sus valores en formato JSON: 'query_usuario', 'accion_sugerida', 'feedback_usuario'.
            """
        )
feedback_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(feedback_prompt),
    HumanMessagePromptTemplate.from_template(""),
])

feedback_pipeline = feedback_prompt | llm 

def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in state_map:
        # Si no existe, crea un nuevo historial de chat en memoria
        state_map[session_id] = InMemoryChatMessageHistory()
        return state_map[session_id]
    response = feedback_pipeline.invoke({"history": state_map[session_id].messages})
    #mydatajson = search_json_in_text(str(response))
    state_map[session_id].add_ai_message(str(response))
    return state_map[session_id]

In [33]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [34]:
reply = pipeline_with_history.invoke(
    {"query": "hola, cual es mi saldo actual?"},
    config={"session_id": "id_123"}
)
print(reply)

mostrar saldo de cuenta

¿desea proceder con esta acción?


In [35]:
reply = pipeline_with_history.invoke(
    {"query": "Si, solo ver mi saldo"},
    config={"session_id": "id_123"}
)
print(reply)

AI: mostrar saldo de cuenta 

¿desea proceder con esta acción?


In [36]:
reply = pipeline_with_history.invoke(
    {"query": "Quiero invertir 5000 dolares en acciones tecnológicas."},
    config={"session_id": "id_123"}
)
print(reply)

AI: invertir 5000 dolares en acciones tecnologicas


In [37]:
reply = pipeline_with_history.invoke(
    {"query": "Si, procede."},
    config={"session_id": "id_123"}
)
print(reply)

Para proporcionar el mejor servicio posible a tu cliente, necesito tomar en cuenta su historial de acciones y responder de manera adecuada.

Dado que el cliente ha solicitado invertir $5000 en acciones tecnológicas, considerando su comportamiento anterior, no veo la acción necesaria para proceder con esta solicitud. El historial anterior muestra una acción para "mostrar saldo de cuenta" pero no proporciona suficiente información sobre cómo está utilizando sus fondos.

Por lo tanto, mi respuesta es:
ninguna


In [40]:
for msg in state_map["id_123"].messages:
    print(f"{msg.type}: {msg.content}")

human: hola, cual es mi saldo actual?
ai: mostrar saldo de cuenta

¿desea proceder con esta acción?
ai: Aquí tienes un ejemplo de código en Python que genera un JSON estructurado según tus especificaciones:

```python
import json

class Message:
    def __init__(self, content, additional_kwargs=None, response_metadata=None):
        self.content = content
        self.additional_kwargs = additional_kwargs if additional_kwargs else {}
        self.response_metadata = response_metadata if response_metadata else {}

class HumanMessage(Message):
    pass

class AIMessage(Message):
    def __init__(self, content, additional_kwargs=None, response_metadata=None):
        super().__init__(content, additional_kwargs, response_metadata)
        self.accion_sugerida = "mostrar saldo de cuenta"

def generar_resumen(historico_conversacion):
    resumen = {
        'query_usuario': "hola, cual es mi saldo actual?",
        'accion_sugerida': 'mostrar saldo de cuenta',
        'feedback_usuario': ""
